In [ ]:
import tweepy 
from tweepy import OAuthHandler # to authenticate Twitter API
from tweepy import Stream 
from tweepy.streaming import StreamListener
import socket 
import json 
import pandas as pd
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Twitter developer Credentials to connect to twitter account
access_token = '1294388428575047689-uQDkJfSiCikiKi1Yi25NvoFh1SKPvT'
access_secret = 'FWBHK9tgHXUzTvIljhwOrCbBn1aIrOD9VLNGigUXbRBzg'
consumer_key = 'HhzYCY5pUaJII2gUAvzqSVyX3'
consumer_secret = 'UqbEdCradZDBEvWCxQCNTwjhJ1szw10iv9JVHmuaAjSZMR49yl'
records_per_file = 5000  # The number of tweets you want to store per file


class TweetsListener(StreamListener):
    # initialized the constructor
    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            # read the Twitter data which comes as a JSON format
            msg = json.loads(data)
            tweet = msg["text"] 
            # the 'text' in the JSON file contains the actual tweet.
            print(msg['text'].encode('utf-8'))
            
            #write to file
            with open("/Users/rashidesai/Downloads/streamoftweetsoutput.csv","a") as fid:
                fid.write(tweet+"\n")
            return True
    
            # the actual tweet data is sent to the client socket
            self.client_socket.send(msg['csv'].encode('utf-8'))
            return True

        except BaseException as e:
            # Error handling
            print("Ahh! Look what is wrong : %s" % str(e))
            return True

    def on_error(self, status):
        print(status)
        return True
    
    def create_dict(list_):
        no_of_tweets = 3000
        dictn =  {k:no_of_tweets for k in list_ }
        return dictn

def sendData(c_socket):
    # authentication
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # twitter_stream will get the actual live tweet data
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    # filter the tweet feeds related to "hospital"
    twitter_stream.filter(track=['hospital'], languages=['en']) 

# create a socket object
s = socket.socket()

# SparkContext(“local[1]”) would not work with Streaming bc 2 threads are required, so we will use local[2]
sc = SparkContext("local[2]", "Twitter Demo")
ssc = StreamingContext(sc, 5) #5 is the batch interval in seconds

# Get local machine name : host and port
host = "127.0.0.1"
port = 9999
lines = ssc.socketTextStream(host, port)

# Bind to the port
s.bind((host, port))
print("Listening on port: %s" % str(port))

# Wait and Establish the connection with client.
s.listen(5)
c, addr = s.accept()

print("Received request from: " + str(addr))

# Keep the stream data available
sendData(c)

ssc.start()
ssc.stop()

Listening on port: 9999
Received request from: ('127.0.0.1', 56321)
b'RT @AGHamilton29: We knew this was coming. Delayed medical treatments and a reduction in preventative visits will lead to deaths. There is\xe2\x80\xa6'
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'RT @TorontoStar: Cancer surgeons in Ontario are reporting a sharp rise in the number of people coming to hospital with advanced cancers. It\xe2\x80\xa6'
b'RT @TorontoStar: Cancer surgeons in Ontario are reporting a sharp rise in the number of people coming to hospital with advanced cancers. It\xe2\x80\xa6'
b'RT @BetoORourke: Just got off the phone w a woman whose dad &amp; mom have covid and are at Providence hospital in El Paso. Her dad is a Korea\xe2\x80\xa6'
b'@MarkJudy19 @ErrolWebber Late term abortions are only used in medical emergencies and usually in the hospital. My s\xe2\x80\xa6 https://t.co/LvXlV2OU5X

b'Never seen an albino obituary poster, meaning death don\xe2\x80\x99t visit them? \xf0\x9f\x98\x82'
b'@drdavidsamadi The US numbers are mostly fake so the Europeans being closer to communism cheat even more.  My docto\xe2\x80\xa6 https://t.co/EhHTWhHcAs'
b'RT @bhaveepatel08: A great opportunity to come and join our forward thinking Paediatric Pharmacy team at Morriston Hospital, Swansea! @UhbB\xe2\x80\xa6'
b'RT @washingtonpost: Texas tries to shore up hospital capacity in El Paso as coronavirus cases surge nationwide https://t.co/JumWBStvVb'
b'RT @StNicholas_ng: St. Nicholas Hospital fully identify with the current situation and our sympathy is with the victims of the unfortunate\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @ImmoralishMe: My 90 year old Granny was sick last week but got better and was sent home from the hospital. She\xe2\x80\x99s still si

b'RT @AlexBerenson: LOCKDOWNS KILL. UNNECESSARY MEDIA-DRIVEN PANIC KILLS.'
b"@kayleighmcenany @realDonaldTrump That's a lot of people not wearing masks. How many of them will get sick, end up\xe2\x80\xa6 https://t.co/DSf8NKNNwH"
b'RT @BuckSexton: People are dying because of shut downs- in this case, cancers that should have been diagnosed earlier. \n\nBut sure, people s\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'RT @NH_India: #India\xe2\x80\x99s first #WorldCup-winning cricket captain #KapilDev was on Sunday discharged from the hospital after undergoing a succ\xe2\x80\xa6'
b'RT @BuckSexton: People are dying because of shut downs- in this case, cancers that should have been diagnosed earlier. \n\nBut sur

b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'How long will that last though, give their kids free education, constant electrity, free healthcare service, these\xe2\x80\xa6 https://t.co/p24vWUdngE'
b'RT @ddale8: As the president says things are going "beautifully" and that we\'re just catching mild cases because we\'re doing so much testin\xe2\x80\xa6'
b'RT @DrEricDing: New\xe2\x80\x94Texas\xe2\x80\x99s epidemic so bad now,  Governor Abbott is requesting the help  of a military hospital to help civilian hospitals\xe2\x80\xa6'
b'RT @JamesSurowiecki: I thought virus was going to virus, no matter what.'
b'RT @YouAreLobbyLud: A thread on racism in COVID management in Australia. First, listen to the interview below from 11 minutes 30 seconds on\xe2\x80\xa6'
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his rel

b'RT @schemaly: Never forget, this wasn\xe2\x80\x99t inevitable. Leaders made choices that mean more people will die. They denied science and infused de\xe2\x80\xa6'
b'#2 for both of my California boys \xf0\x9f\x96\xa4'
b'RT @schemaly: Never forget, this wasn\xe2\x80\x99t inevitable. Leaders made choices that mean more people will die. They denied science and infused de\xe2\x80\xa6'
b'RT @AvaGG: Go donate!!'
b"@AishaYesufu @ChiefOjukwu @fkabudu @mrmacaronii @SavvyRinu\nPls send hospital number, we'll replace phone\nThere'll A\xe2\x80\xa6 https://t.co/9Os8SimQCf"
b'RT @berniespofforth: At the peak of COVID 40% of NHS beds were empty. 4 times the normal amount.'
b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'I wonder how 

b'RT @manuelfherrador: \xc2\xbfA qu\xc3\xa9 os suena Ayuso traducida al ingl\xc3\xa9s?\n\n"It\'s a great hospital. It\'s gonna be the greatest hospital, with more tha\xe2\x80\xa6'
b'RT @StNicholas_ng: St. Nicholas Hospital fully identify with the current situation and our sympathy is with the victims of the unfortunate\xe2\x80\xa6'
b'RT @SRuhle: There are some very large Trump donors who sit on hospital boards, fund entire hospital systems, endow medical schools.\nHow do\xe2\x80\xa6'
b'RT @kurteichenwald: My sister received her bill for the time her husband spent in the hospital with COVID, including almost two weeks in th\xe2\x80\xa6'
b'@NBCNews Trump came to Dayton to meet the survivors of the Oregon District shootings. He left the hospital as his M\xe2\x80\xa6 https://t.co/6Y1hHWyXVA'
b'RT @ComradeKalEl: I\'ve been putting it off, because I really don\'t like to do this, but one of my mom\'s "friends" stole 600 bucks from us b\xe2\x80\xa6'
b'RT @Kelsea67591086: There are more people

b'RT @exjon: An "unintended consequence" some of us warned about in March. Monomaniacal focus on COVID is a threat to public health. https://\xe2\x80\xa6'
b'RT @LeoVaradkar: Hospital overcrowding is at a record low for this time of year and has been for months. We have beds free in most hospital\xe2\x80\xa6'
b'RT @ddale8: As the president says things are going "beautifully" and that we\'re just catching mild cases because we\'re doing so much testin\xe2\x80\xa6'
b'RT @DrEricDing: New\xe2\x80\x94Texas\xe2\x80\x99s epidemic so bad now,  Governor Abbott is requesting the help  of a military hospital to help civilian hospitals\xe2\x80\xa6'
b'RT @jhstlouis: Actually just don\xe2\x80\x99t change a patient\xe2\x80\x99s BP meds in the hospital for any reason unless they came in with HTN urgency/emergency\xe2\x80\xa6'
b'RT @ben_johns: MPs shouldn\xe2\x80\x99t get expenses, especially when teachers are spending their own money on resources for children and NHS staff ar\xe2\x80\xa6'
b'RT @StNicho

b'RT @ChidiOdinkalu: Folorunsho Olabisi was listed missing following the #LekkiMassacre. \n\nHis body was found by his relatives at the mortuar\xe2\x80\xa6'
b'@daminister14 Guy man, for your own sanity or reputation, this man will lead you into the pit o. Stopeet \xf0\x9f\x98\x82\nOf all t\xe2\x80\xa6 https://t.co/dcnTfvXAAT'
b'RT @manuelfherrador: \xc2\xbfA qu\xc3\xa9 os suena Ayuso traducida al ingl\xc3\xa9s?\n\n"It\'s a great hospital. It\'s gonna be the greatest hospital, with more tha\xe2\x80\xa6'
b'RT @_lauryncalo22: all my people in the valley/LA, if u see bro, say sum. My cousin is in the hospital without an eye &amp; other serious healt\xe2\x80\xa6'
b'RT @avogroovy: \xe2\x80\x9cThe soldiers, who were deployed from 65 Battalion, Bonny Camp, Victoria Island, Lagos, told some of the victims\xe2\x80\x99 relatives\xe2\x80\xa6'
b'@BasedLAComedian Something that also makes me a rage are hospital ER covid restrictions!!! You give me a number to\xe2\x80\xa6 https://t.co/uUCCz7rlyb'
b"R